In [ ]:
import os
import json
import glob
from tqdm.notebook import tqdm

import torch
from transformers import (
    AutoModelForCausalLM, 
    AutoModelForCausalLM as BaseCausalLM,
    AutoTokenizer, 
    BitsAndBytesConfig, 
    pipeline
)
from peft import PeftModel
from datasets import Dataset
from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead

In [2]:
# Load Hugging Face API token from file and set it as an environment variable.
keys_path = "../keys.json"
with open(keys_path, "r") as f:
    keys = json.load(f)
os.environ["HF_TOKEN"] = keys["huggingface_access_token"]

print("Keys loaded successfully.")


Keys loaded successfully.


In [3]:
# Define model name and cache directory.
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
scratch_dir = "../models"
device = torch.device("cuda:0")

# Create the quantization configuration.
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load the tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=scratch_dir)

# --- Load the base model and then wrap it with your LoRA adapter ---
# First, load the base model in 4-bit.
# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     cache_dir=scratch_dir,
#     device_map={"": 0},
#     quantization_config=quantization_config
# )

# Load your fine-tuned LoRA adapter.
# (This returns a wrapped model using PEFT.)
# sft_model = PeftModel.from_pretrained(base_model, "./lora_mistral_adapter_mindmaps_v1")
# print("SFT model with LoRA adapter loaded.")

In [ ]:
# --- For PPO, load a value-headed model and a frozen reference model. ---
# We assume you want your SFT weights (with adapter) to be available in the value-headed model.
# In many setups you would merge LoRA adapter weights into a value-headed model.
# Here, we load the value-headed model from scratch and assume the adapter is merged.
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir=scratch_dir,
    device_map={"": 0},
    quantization_config=quantization_config
)

# Load LoRA adapter into base model
peft_model = PeftModel.from_pretrained(base_model, "./lora_mistral_adapter_mindmaps_v1")

# Wrap it with a value head for PPO
ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(peft_model)

# Prepare reference model (no LoRA, just same base weights)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_name,
    cache_dir=scratch_dir,
    device_map={"": 0},
    quantization_config=quantization_config
)
ref_model.eval()

print("Value-headed model and reference model set for PPO training.")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Value-headed model and reference model set for PPO training.


In [5]:
# Load training entries.
train_data_path = "../data/tqa_train_val_test/train/tqa_v1_train.json"
with open(train_data_path, "r", encoding="utf-8") as f:
    train_entries = json.load(f)
print(f"Loaded {len(train_entries)} training entries.")

# Function to load ground truth mind maps from a given folder.
def load_gt_mindmaps(folder):
    gt = {}
    for file_path in glob.glob(os.path.join(folder, "*.txt")):
        example_id = os.path.splitext(os.path.basename(file_path))[0]
        with open(file_path, "r", encoding="utf-8") as f:
            gt[example_id] = f.read().strip()
    return gt

gt_mindmaps_v1 = load_gt_mindmaps("../data/tqa_train_val_test/train/mindmap_text_v1")
gt_mindmaps_v2 = load_gt_mindmaps("../data/tqa_train_val_test/train/mindmap_text_v2")
print(f"Loaded {len(gt_mindmaps_v1)} GT mind maps from v1 and {len(gt_mindmaps_v2)} from v2.")



Loaded 666 training entries.
Loaded 666 GT mind maps from v1 and 666 from v2.


In [ ]:

pairwise_eval_path = "../data/tqa_train_val_test/train/pairwise_evaluations.json"

with open(pairwise_eval_path, "r", encoding="utf-8") as f:
    evaluations = json.load(f)

preferred_mindmaps = {}
for record in evaluations:
    # Use the filename as the example ID.
    filename = record["filename"]
    example_id = os.path.splitext(filename)[0]
    
    # Determine the preferred output based on the evaluation.
    preferred_output = record["evaluation"]["preferred_output"]
    if preferred_output == "Output A":
        preferred_path = record["mind_map_v1_path"]
    elif preferred_output == "Output B":
        preferred_path = record["mind_map_v2_path"]
    else:
        preferred_path = ""  # Fallback if preferred_output is not recognized.
    
    # Read the preferred mind map content.
    if preferred_path and os.path.exists(preferred_path):
        with open(preferred_path, "r", encoding="utf-8") as fp:
            mind_map_text = fp.read().strip()
    else:
        mind_map_text = ""
    
    preferred_mindmaps[example_id] = mind_map_text

print(example_id,"\n",preferred_mindmaps[example_id])

mermaid_code_L_0547 
 mindmap
  Human Causes of Extinction
    Causes of Extinction
      Primary Cause
        Human-induced extinctions
      Greatest Contributor
        Habitat destruction
    Global Climate Change
      Definition
        Long-term change in climate
      Effects on Organisms
        Hazardous to survival
    Global Warming
      Definition
        Increase in Earth's average temperature
      Impact on Species
        Destruction of habitats
        Stress from brief climate changes
    Pollution
      Types
        Chemicals
        Noise
        Heat
        Light
      Effects on Organisms
        Threats to survival
        Example: DDT and peregrine falcons
        Water pollution
        Soil contamination
    Human Population Growth
      Current Statistics
        Over 7 billion in 2011
        Projected 8-9 billion by mid-century
      Resource Demand
        Need for housing, food, water
      Contribution to Extinction
        Increased global warming


In [7]:
def extract_textbook_text_with_image_explainations(entry,):
    """
    Extract textbook texts and any image-based explanations from a test entry.
    This function consolidates lesson names, section texts, and image explanations.
    """
    texts = []
    lesson_name = entry.get("lessonName", "")
    if lesson_name:
        texts.append(f"Lesson: {lesson_name.strip()}")
    
    base_dir = "../data/tqa_train_val_test/train/"
    
    def extract_content_and_explanations(section):
        section_texts = []
        content = section.get("content", {})
        text = content.get("text", "")
        if text:
            section_texts.append(text.strip())
        figures = content.get("figures", [])
        for fig in figures:
            caption = fig.get("caption", "").strip()
            image_path = fig.get("imagePath", "")
            if image_path:
                if "textbook_images" in image_path:
                    explanation_folder = "textbook_images_llava_captions"
                elif "teaching_images" in image_path:
                    explanation_folder = "teaching_images_llava_captions"
                else:
                    explanation_folder = None
                if explanation_folder:
                    file_name = os.path.basename(image_path)
                    base_name = os.path.splitext(file_name)[0]
                    explanation_file = os.path.join(base_dir, explanation_folder, base_name + ".txt")
                    if os.path.exists(explanation_file):
                        with open(explanation_file, 'r', encoding="utf-8") as f:
                            explanation = f.read().strip()
                        if explanation:
                            combined_text = ""
                            if caption:
                                combined_text += f"Image Caption: {caption}. "
                            combined_text += f"Image Explanation: {explanation}"
                            section_texts.append(combined_text)
        return section_texts

    # Extract texts from adjunct topics and main topics.
    adjunct_topics = entry.get("adjunctTopics", {})
    for topic in adjunct_topics.values():
        texts.extend(extract_content_and_explanations(topic))
    
    topics = entry.get("topics", {})
    for topic in topics.values():
        texts.extend(extract_content_and_explanations(topic))
    
    return texts

In [8]:
def build_training_prompt(entry):
    """Build the complete prompt with a fixed instruction and the extracted textbook text."""
    extracted_texts = extract_textbook_text_with_image_explainations(entry)
    combined_text = "\n\n".join(extracted_texts)
    prompt = (
         "Instruction: Generate a mind map in Mermaid syntax for the following textbook text. "
         "The mind map should capture the main topics and their subtopics clearly in a hierarchical structure. "
         "Ensure that there is exactly one central (root) node, which is the only root, and attach all other topics "
         "as subtopics of this central node. Do NOT include a 'root' node with a generic label; instead, use a relevant central topic. "
         "Also, do NOT include any theme directives such as %%{init: {\"theme\": \"default\"}}%%, any parentheses or any extraneous formatting. "
         "The mind map should be self-explanatory so that by reading it, a user can understand the key content and structure "
         "of the text document. Keep the Mermaid syntax minimal so it renders correctly, and be concise.\n\n"
         f"Input: {combined_text}\nOutput:"
    )
    return prompt

In [ ]:
training_examples = []

print("Sample training entry:", train_entries[0])

for entry in train_entries:
    # Use the "globalID" field from the training entry.
    example_id = entry.get("globalID", "").strip()
    if not example_id:
        continue

    # If the training entry's ID doesn't already include the prefix, add it.
    if not example_id.startswith("mermaid_code_"):
        lookup_id = f"mermaid_code_{example_id}"
    else:
        lookup_id = example_id

    # Retrieve the preferred mind map using the lookup_id.
    preferred = preferred_mindmaps.get(lookup_id, "")
    
    # Build the prompt using your helper function.
    prompt = build_training_prompt(entry)
    
    # Only include the entry if a preferred mind map is found.
    if preferred:
        training_examples.append({
            "example_id": example_id,
            "prompt": prompt,
            "preferred": preferred
        })

print(f"Prepared {len(training_examples)} training examples.")


Sample training entry: {'adjunctTopics': {'Apply Concepts': {'content': {'figures': [], 'mediaLinks': [], 'text': '5. A glacier is melting. What are all of the scientists you can think of who might be involved in studying this glacier? What would each of them do? '}, 'orderID': 't_12'}, 'Introduction': {'content': {'figures': [{'caption': 'FIGURE 1.10 Earth as seen from Apollo 17.', 'imagePath': 'textbook_images/earth_science_and_its_branches_20010.png'}], 'mediaLinks': [], 'text': 'Earth Science is the study of all aspects of our planet Earth. Earth Science is not just about the molten lava, icy mountain peaks, steep canyons and towering waterfalls of the continents. Earth Science includes the atmosphere and oceans. The field also looks out into the solar system, galaxy, and universe. Earth scientists seek to understand the beautiful planet on which we depend (Figure 1.10). Different branches of Earth Science study one particular part of Earth. Since all of the branches are connected,

In [10]:
train_dataset = Dataset.from_list([
    {
        "query": example["prompt"],
        "example_id": example["example_id"]
    }
    for example in training_examples
])
print("Hugging Face training dataset created.")
print(train_dataset)

Hugging Face training dataset created.
Dataset({
    features: ['query', 'example_id'],
    num_rows: 664
})


In [11]:
def load_api_key(path: str = "../keys.json") -> None:
    """Load OpenAI API key from a JSON file into environment."""
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    os.environ["OPENAI_API_KEY"] = data.get("openai_api_key")

In [12]:
load_api_key()

In [ ]:
import requests
import re

def compute_reward_gpt4o(ground_truth: str, prediction: str) -> float:
    """
    Compute a reward for the generated mind map by comparing it to the ground truth mind map
    using GPT-4O-mini.
    
    The function calls the GPT-4O-mini API with an evaluation prompt. It expects a JSON
    response containing "cognitive_score" and "structural_score" (values 1 to 7). We then 
    compute the average score and normalize it so that a score of 1 corresponds to 0.0 reward and 
    a score of 7 corresponds to 1.0 reward.
    
    Returns:
         normalized_reward (float): Overall reward normalized between 0 and 1.
    """
    prompt = f"""
You are an expert evaluator of mind maps. Evaluate the quality of the generated mind map relative to the ground truth.
Consider the following dimensions:

1. **Cognitive Indicators**: How easily can a user understand and mentally process the information? 
   Evaluate clarity, flow of ideas, and thematic relevance.

2. **Structural Indicators**: How well-organized and logically complete is the mind map? 
   Evaluate grouping of ideas, coverage of key concepts, level of detail, and consistency of hierarchy.

Scoring Instructions:
- Use a Likert scale from 1 (much worse) to 7 (much better).
- A score of 4 indicates the generated mind map is equal in quality to the ground truth.

Return your evaluation as a JSON object with the keys:
  "cognitive_score": int,
  "structural_score": int

Ground Truth Mind Map:
{ground_truth}

Generated Mind Map:
{prediction}
    """
    
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("OPENAI_API_KEY not set — run load_api_key() first.")

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "system", "content": "You are an expert evaluator of mind maps."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    response.raise_for_status()
    
    # Retrieve and clean the returned text.
    text = response.json()["choices"][0]["message"]["content"].strip()
    
    # Try parsing the text as JSON.
    try:
        evaluation = json.loads(text)
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", text, re.S)
        if match:
            evaluation = json.loads(match.group())
        else:
            raise ValueError("Could not parse JSON from API response.")

    # Extract the scores
    cognitive = evaluation.get("cognitive_score", 4)
    structural = evaluation.get("structural_score", 4)
    
    # Compute the average score.
    avg_score = (cognitive + structural) / 2.0
    # Normalize: (score - 1) / (7 - 1) so that 1 becomes 0.0 and 7 becomes 1.0.
    normalized_reward = (avg_score - 1) / 6.0
    
    # Print the evaluation and computed reward for debugging.
    print("Evaluation:", evaluation)
    print("Average score:", avg_score, "Normalized reward:", normalized_reward)
    
    return normalized_reward

In [14]:
from transformers import AutoConfig, GenerationConfig
# Load generation config
gen_config = GenerationConfig.from_pretrained(model_name)

# Attach to model manually
ppo_model.generation_config = gen_config
ppo_model.base_model_prefix = "pretrained_model"

device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
ppo_model = ppo_model.to(device)
ppo_model.device = device 

In [15]:
from torch import nn
class DummyRewardModel(nn.Module):
    def forward(self, *args, **kwargs):
        return torch.tensor([0.0])

dummy_reward_model = DummyRewardModel().to(device)

In [ ]:
ppo_config = PPOConfig(
    model_name=None,          
    learning_rate=1e-5,
    batch_size=1,  # must be divisible by mini_batch_size * gradient_accumulation_steps
    mini_batch_size=1,
    log_with="tensorboard",
    project_kwargs={"logging_dir": "./ppo_logs"}
)
ppo_trainer = PPOTrainer(
    config=ppo_config,
    tokenizer=tokenizer,
    model=ppo_model,
    ref_model=ref_model,
)
print("PPO trainer initialized with training dataset.")


PPO trainer initialized with training dataset.


/home/tsubrama/.local/lib/python3.11/site-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(
/home/tsubrama/.local/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(
/home/tsubrama/.local/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:273: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [ ]:
import gc
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence

# Ensure tokenizer has a pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

num_epochs = 1  # Adjust as needed
print("Starting PPO training loop...\n")

for epoch in range(num_epochs):
    # print(f"Epoch {epoch + 1}/{num_epochs}")
    
    for example in tqdm(training_examples, desc=f"Epoch {epoch + 1}"):
        prompt = example["prompt"]
        preferred = example["preferred"]

        # Tokenize prompt and move to device
        query_inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=5000)
        query_inputs = {k: v.to(device) for k, v in query_inputs.items()}
        query_tensors = query_inputs["input_ids"]

        # Generate response using model
        with torch.inference_mode():
            output_ids = ppo_model.generate(
                **query_inputs,
                max_new_tokens=300,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Extract generated response only (excluding prompt)
        response_tensors = output_ids[:, query_tensors.shape[-1]:]

        # Decode generated response only
        generated_mindmap = tokenizer.decode(response_tensors[0], skip_special_tokens=True)

        # Compute reward from preferred vs. generated output
        reward = compute_reward_gpt4o(preferred, generated_mindmap)
        reward_tensor = torch.tensor([reward], device=device)

        # PPO step
        stats = ppo_trainer.step(
            queries=[query_tensors[0]],
            responses=[response_tensors[0]],
            scores=[reward_tensor]
        )

        # # Logging
        # print(f"Example ID: {example['example_id']}")
        # print(f"Reward: {reward}")
        # print("PPO update stats:", stats, "\n")

        # Memory cleanup
        del query_inputs, query_tensors, output_ids, response_tensors, reward_tensor
        torch.cuda.empty_cache()
        gc.collect()

    # print("------------------------------------------------------\n")

print("PPO training complete.")

Starting PPO training loop...



Epoch 1:   0%|          | 0/664 [00:00<?, ?it/s]

Evaluation: {'cognitive_score': 5, 'structural_score': 6}
Average score: 5.5 Normalized reward: 0.75


/home/tsubrama/.local/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1309: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  std_scores = data["scores"].std()
/home/tsubrama/.local/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1336: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  stats["tokens/queries_len_std"] = torch.std(query_lens).cpu().numpy().item()
/home/tsubrama/.local/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1339: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/

Evaluation: {'cognitive_score': 5, 'structural_score': 4}
Average score: 4.5 Normalized reward: 0.5833333333333334


...

Evaluation: {'cognitive_score': 5, 'structural_score': 6}
Average score: 5.5 Normalized reward: 0.75


Epoch 1: 100%|█████████▉| 663/664 [4:05:13<00:17, 17.01s/it]

Evaluation: {'cognitive_score': 5, 'structural_score': 6}
Average score: 5.5 Normalized reward: 0.75


Epoch 1: 100%|██████████| 664/664 [4:05:27<00:00, 22.18s/it]

PPO training complete.


In [18]:
output_model_dir = "./ppo_trained_model"
os.makedirs(output_model_dir, exist_ok=True)

ppo_model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
print(f"Trained model saved to {output_model_dir}")


Trained model saved to ./ppo_trained_model


## Inference

In [ ]:
import os
import torch
from transformers import AutoTokenizer, GenerationConfig
from trl import AutoModelForCausalLMWithValueHead

# the directory where PPO-trained model and tokenizer are saved.
model_dir = "./ppo_trained_model"
scratch_dir = "../models"
device = torch.device("cuda:0")

# Disable remote downloads by checking that the directory exists.
if not os.path.exists(model_dir):
    raise FileNotFoundError(f"The directory '{model_dir}' does not exist. "
                            "Please ensure your model is saved locally.")

# Load the tokenizer from the local directory.
tokenizer = AutoTokenizer.from_pretrained(model_dir, cache_dir=scratch_dir)

# Load the model with its value head from the local directory.
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_dir, cache_dir=scratch_dir)
model.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

AutoModelForCausalLMWithValueHead(
  (pretrained_model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): MistralForCausalLM(
        (model): MistralModel(
          (embed_tokens): Embedding(32768, 4096)
          (layers): ModuleList(
            (0-31): 32 x MistralDecoderLayer(
              (self_attn): MistralSdpaAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4096, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()


In [2]:
combined_text="""
Motion suggests life, and as such, motion is a dimension we add to our
designs to make them more dynamic and engaging. Animation is a special
type of design form which we have created to help us take static designs
into more media-rich and interactive contexts. Aspecific type of animated 
content that we frequently create is the animated logo. 

Animation allows logos, which have been defined as the “visual figureheads" 
of brands [25], to better integrate within videos, livestreams, websites,
and social media.

A well-executed animation can quickly engage an audience, introduce the 
brand or individual online, and elevate content to have more visual interest.

Authoring an animated logo is challenging. Logos are often more than just a 
pairing of icon with text. Because they can have different layouts, layers,
color, and typography, they can take on great variety and be complex artifacts
to animate.

For a novice designer, it can be difficult to understand which design elements 
should be animated, in what sequence, and how to build up compelling and
believable motion. There are many facets of motion to consider such as speed,
timing, positioning, duration, easing, and motion personality
(e.g. a playful bounce vs. a strong entrance). 

Additionally, when logos have more design elements, designers also have to
understand how groups of elements can synchronize to coordinate motion and 
orchestrate a visual flow. While there is a great demand for animated content, 
it is diffi cult for people outside of motion design to develop this kind of
expertise. 

Design tools such as Adobe Express, Canva, and Figma often provide 
solutions in the form of animated templates and au tomatic animation 
techniques [10, 12, 13]. Templates pre-populate logo layouts with animations
that users can customize. They il lustrate how users can apply motion presets
(e.g. slide, flicker, or fade) onto logo elements to create professional-looking
animations. However, templates do not always adapt to every use case. 

When users make edits (e.g. add/remove/replace elements) to customize logo
templates, they can easily break the seamless and professional look the
templates were originally packaged with. An alternative to templates are
automatic animation techniques, which globally apply rules and heuristics to
animate canvases [12]. 

For example, all elements on a page can be directed to slide in from one side 
or 1Video: https://youtu.be/Jo9opkMH7iY 2Project P
age: https://vivian-liu.com/#/logomotion sequentially fade into place.
While templates and automatic tech niques can get users to a starting point fast,
neither solution works with a recognition of the user’s content,
which is something that can be enabled by emerging technologies.
Large language models (LLMs) present the potential for content aware animation.

They can generate animation code that is specific to the design elements and their 
layout on the canvas. Code is a text representation that is often used to drive animation [18, 33, 53], because it can concisely specify how elements interact over time and space on a canvas. Because LLMs encode a vast amount of world knowledge, they can draw upon actions and activities related to the content being animated and generate a near infinite number of animations. This open-ended generative capacity can go beyond the scope of what templates, presets, and rule-based techniques usually cover. Recent advancements have made LLMs more multimodal, such that they can take in both text and image as inputs, and provide visually-grounded responses. This make LLMs more applicable in domains like animation where a visual understanding of the canvas matters. It opens upthepotentialforuserstoprovideimagesoftheir layout to an LLM and receive animations tailored to their layout and design elements. For example, if a novice designer wanted to animate a taxi, they could use an LLM to generate code to drive a taxi onto the canvas. This code could translate the taxi object along the x-axis before easing it into the center of the canvas to imply a stop-and-go motion befitting of taxis. In this paper, we present LogoMotion, an LLM-based method that automatically animates static layouts in a content-aware way. LogoMotion generates code in a two-stage approach involving visually-grounded program synthesis and program repair. The first stage introduces multimodal LLM operators that take in visual context andhandlethe1)constructionofatextrepresentationofthe canvas, 2) conceptual grouping of elements, and 3) implementation of animation code. The second stage of our approach introduces a technique for visually-grounded program repair, which helps LLMs check what they have generated against the original layout and debug differences in a targeted layer-wise fashion. Our contributions are as follows: • LogoMotion, an LLM system that uses visually-grounded code generation to automatically generate logo animations from a PDF. The system identifies the visual content in each layer, infers the primary and secondary elements, and cre ates groups of elements. Based on this, the system suggests a design concept (in text) and uses the LLM to generate ani mation code. Users can optionally improve the animation by editing or adding their own design concept. • Visually-grounded program repair, a mechanism that lets the LLM automatically detect and debug visual errors within its generated animation code, creating a feedback loop between LLM-generated code and its visual outputs. • Atechnical evaluation of 276 animations showing that com pared to Canva Magic Animate and an ablated version of the system (without stages for hierarchy analysis and de sign concept suggestions), the full pipeline of LogoMotion produces animations that are more content-aware. LogoMotion: Visually Grounded Code Generation for Content-Aware Animation Woodstock ’18, June 03-05, 2018, Woodstock, NY • Aqualitative evaluation of novice users showing that Logo Motion is able to quickly achieve their desired animation with minimal reprompting.
"""
prompt_text = (
     "Instruction: Generate a mind map in Mermaid syntax for the following textbook text. "
     "The mind map should capture the main topics and their subtopics clearly in a hierarchical structure. "
     "Ensure that there is exactly one central (root) node, which is the only root, and attach all other topics "
     "as subtopics of this central node. Do NOT include a 'root' node with a generic label; instead, use a relevant central topic. "
     "Also, do NOT include any theme directives such as %%{init: {\"theme\": \"default\"}}%%, any parentheses or any extraneous formatting. strictly no ( or ) "
     "The mind map should be self-explanatory so that by reading it, a user can understand the key content and structure "
     "of the text document. Keep the Mermaid syntax minimal so it renders correctly, and be concise.\n\n"
     f"Input: {combined_text}\n"
     "Output:"
)


In [ ]:

# Load the generation configuration if it exists.
try:
    gen_config = GenerationConfig.from_pretrained(model_dir)
    model.generation_config = gen_config
except EnvironmentError:
    print("No generation_config.json found; using default generation settings.")

# Tokenize the prompt and move inputs to the device.
inputs = tokenizer(prompt_text, return_tensors="pt", truncation=True, padding=True)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate output from the model.
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=300,   # adjust as needed
        do_sample=True,       # sample-based generation for diversity
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

# The generated output includes both the prompt tokens and new tokens. Exclude the prompt.
prompt_len = inputs["input_ids"].shape[1]
generated_ids = output_ids[:, prompt_len:]

# Decode the generated tokens to text.
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Generated Mind Map:")
print(generated_text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


No generation_config.json found; using default generation settings.
Generated Mind Map:
mindmap
  Animation
    Definition
      Motion suggests life
      Dynamic and engaging content
      Integrates with media-rich contexts
      Enhances visual interest
    Animated Logo
      Visual figurehead of brands
      Challenging to author
      Design elements to consider
        Speed
        Timing
        Positioning
        Duration
        Easing
        Motion personality
    Design Tools
      Adobe Express
      Canva
      Figma
      Automatic animation techniques
        Sequential motion
        Global rules
        Limited adaptation
    Emerging Technologies
      Large Language Models (LLMs)
        Content-aware animation
        Generative capacity beyond templates
        Code for animation
    LogoMotion
      LLM-based method
      Visually-grounded code generation
      Program synthesis and repair
    Technical Evaluation
      Content-aware animations
      Comparis

## Evaluation

In [6]:
def extract_textbook_text_with_image_explainations(entry):
    texts = []
    lesson_name = entry.get("lessonName", "")
    if lesson_name:
        texts.append(f"Lesson: {lesson_name.strip()}")
    
    base_dir = "../data/tqa_train_val_test/train/"
    
    def extract_content_and_explanations(section):
        section_texts = []
        content = section.get("content", {})
        text = content.get("text", "")
        if text:
            section_texts.append(text.strip())
        figures = content.get("figures", [])
        for fig in figures:
            caption = fig.get("caption", "").strip()
            image_path = fig.get("imagePath", "")
            if image_path:
                if "textbook_images" in image_path:
                    explanation_folder = "textbook_images_llava_captions"
                elif "teaching_images" in image_path:
                    explanation_folder = "teaching_images_llava_captions"
                else:
                    explanation_folder = None
                if explanation_folder:
                    file_name = os.path.basename(image_path)
                    base_name = os.path.splitext(file_name)[0]
                    explanation_file = os.path.join(base_dir, explanation_folder, base_name + ".txt")
                    if os.path.exists(explanation_file):
                        with open(explanation_file, 'r', encoding="utf-8") as f:
                            explanation = f.read().strip()
                        if explanation:
                            combined_text = ""
                            if caption:
                                combined_text += f"Image Caption: {caption}. "
                            combined_text += f"Image Explanation: {explanation}"
                            section_texts.append(combined_text)
        return section_texts

    adjunct_topics = entry.get("adjunctTopics", {})
    for topic in adjunct_topics.values():
        texts.extend(extract_content_and_explanations(topic))
    
    topics = entry.get("topics", {})
    for topic in topics.values():
        texts.extend(extract_content_and_explanations(topic))
    
    return texts


In [10]:
def create_test_prompt(entry):
    """
    Builds the test prompt using your extraction function and a fixed instruction.
    This is similar to your training prompt but leaves out any ground truth output.
    """
    extracted_texts = extract_textbook_text_with_image_explainations(entry)
    combined_text = "\n\n".join(extracted_texts)
    prompt = (
         "Instruction: Generate a mind map in Mermaid syntax for the following textbook text. "
         "The mind map should capture the main topics and their subtopics clearly in a hierarchical structure. "
         "Ensure that there is exactly one central (root) node, which is the only root, and attach all other topics "
         "as subtopics of this central node. Do NOT include a 'root' node with a generic label; instead, use a relevant central topic. "
         "Also, do NOT include any theme directives such as %%{init: {\"theme\": \"default\"}}%%, any parentheses or any extraneous formatting. strictly no ( or ) "
         "The mind map should be self-explanatory so that by reading it, a user can understand the key content and structure "
         "of the text document. Keep the Mermaid syntax minimal so it renders correctly, and be concise.\n\n"
         f"Input: {combined_text}\n"
         "Output:"
    )
    return prompt

In [11]:
# ---------------------------
# Inference on Test Data
# ---------------------------
import json
# Load your test dataset.
test_data_path = "../data/tqa_train_val_test/test/tqa_v2_test.json"
with open(test_data_path, "r", encoding="utf-8") as f:
    test_entries = json.load(f)

# Print a sample prompt for inspection.
sample_entry = test_entries[0]
sample_prompt = create_test_prompt(sample_entry)
print("=== Sample Prompt ===")
print(sample_prompt)
print("=====================")

# Tokenize the prompt.
inputs = tokenizer(sample_prompt, return_tensors="pt", truncation=True, padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate the model's output.
# Adjust max_new_tokens or other parameters as needed.
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

# Remove the input prompt tokens to extract only the generated portion.
prompt_length = inputs["input_ids"].shape[1]
generated_ids = output_ids[:, prompt_length:]
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("=== Generated Mind Map ===")
print(generated_text)
print("==========================")

=== Sample Prompt ===
Instruction: Generate a mind map in Mermaid syntax for the following textbook text. The mind map should capture the main topics and their subtopics clearly in a hierarchical structure. Ensure that there is exactly one central (root) node, which is the only root, and attach all other topics as subtopics of this central node. Do NOT include a 'root' node with a generic label; instead, use a relevant central topic. Also, do NOT include any theme directives such as %%{init: {"theme": "default"}}%%, any parentheses or any extraneous formatting. strictly no ( or ) The mind map should be self-explanatory so that by reading it, a user can understand the key content and structure of the text document. Keep the Mermaid syntax minimal so it renders correctly, and be concise.

Input: Lesson: the nature of science

3. Write five questions that would get a friend interested in exploring the natural world. 4. A scientist was studying the effects of oil contamination on ocean sea

In [ ]:
# ---------------------------
# Batch Processing: Save All Predictions
# ---------------------------
from tqdm import tqdm
dataset_folder = os.path.dirname(test_data_path)
predictions_folder = os.path.join(dataset_folder, "test_predictions_rlhf")
os.makedirs(predictions_folder, exist_ok=True)

for i, entry in tqdm(enumerate(test_entries), total=len(test_entries), desc="Generating predictions"):
    prompt = create_test_prompt(entry)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    prompt_length = inputs["input_ids"].shape[1]
    generated_ids = output_ids[:, prompt_length:]
    full_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    if "Output:" in full_text:
        prediction = full_text.split("Output:")[1].strip()
    else:
        prediction = full_text.strip()
    
    # Determine a unique identifier for the file name.
    identifier = entry.get("globalID", f"entry_{i}")
    if not identifier or not identifier.strip():
        lesson_name = entry.get("lessonName", "")
        identifier = lesson_name.strip().replace(" ", "_") if lesson_name.strip() else f"entry_{i}"
    # Sanitize the identifier for safe filenames.
    identifier = "".join([c for c in identifier if c.isalnum() or c in "_-"])
    
    output_file = os.path.join(predictions_folder, f"prediction_{identifier}.txt")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(prediction)
        
print(f"Saved individual predictions to folder: {predictions_folder}")

Generating predictions: 100%|██████████| 210/210 [1:08:06<00:00, 19.46s/it]

Saved individual predictions to folder: ../data/tqa_train_val_test/test/test_predictions_rlhf


In [ ]:
import os
from tqdm import tqdm

dataset_folder = os.path.dirname(test_data_path)
predictions_folder = os.path.join(dataset_folder, "test_predictions_rlhf")
os.makedirs(predictions_folder, exist_ok=True)

for i, entry in tqdm(enumerate(test_entries), total=len(test_entries), desc="Generating predictions"):
    prompt = create_test_prompt(entry)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    prompt_length = inputs["input_ids"].shape[1]
    generated_ids = output_ids[:, prompt_length:]
    full_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    if "Output:" in full_text:
        prediction = full_text.split("Output:")[1].strip()
    else:
        prediction = full_text.strip()
    
    # Determine a unique identifier for the file name.
    identifier = entry.get("globalID", f"entry_{i}")
    if not identifier or not identifier.strip():
        lesson_name = entry.get("lessonName", "")
        identifier = lesson_name.strip().replace(" ", "_") if lesson_name.strip() else f"entry_{i}"
    # Sanitize the identifier for safe filenames.
    identifier = "".join([c for c in identifier if c.isalnum() or c in "_-"])
    
    output_file = os.path.join(predictions_folder, f"prediction_{identifier}.txt")
    
    # Check if the file already exists; if so, skip saving.
    if os.path.exists(output_file):
        continue
    
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(prediction)
        
print(f"Saved individual predictions to folder: {predictions_folder}")
